In [1]:
!pip install Keras

In [2]:
!pip install pymorphy2

     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 8.2 MB 8.3 MB/s 


In [3]:
import keras

In [4]:
!pip install stop_words

  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32912 sha256=d6383d27e81f2a93d00ccfee53d0424ac2586aaf82804c80246fffb75bb8228e
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [5]:
from keras.models import Model
from keras import layers
from keras import Input
from pymorphy2 import MorphAnalyzer
import pickle
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words

In [6]:
def preprocess_txt(morpher, line):
#    sw = set(get_stop_words("ru"))
    exclude = set(string.punctuation)
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
#    spls = [i for i in spls if i not in sw and i != ""]
    return spls


In [7]:
path = '/content/sample_data/paraphrases.xml'

from lxml import objectify
import pandas as pd

xml = objectify.parse(path)
root = xml.getroot()
root.getchildren()[1]

<Element corpus at 0x7f45f2328c30>

In [8]:
root = xml.getroot()
root = root.getchildren()[1]
#<value name="id">1</value>
#<value name="id_1">201</value>
#<value name="id_2">8159</value>
#<value name="text_1">Полицейским разрешат стрелять на поражение по гражданам с травматикой.</value>
#<value name="text_2">Полиции могут разрешить стрелять по хулиганам с травматикой.</value>
#<value name="jaccard">0.65</value>
#<value name="class">0</value>

df = pd.DataFrame(columns=('id','id_1', 'id_2', 'text_1', 'text_2', 'jaccard', 'class'))

for i in range(0,len(root.getchildren())):
    obj = root.getchildren()[i].getchildren()
    row = dict(zip(['id','id_1', 'id_2', 'text_1', 'text_2', 'jaccard', 'class'], [obj[0].text, obj[1].text, obj[2].text, obj[3].text, obj[4].text, obj[5].text, obj[6].text]))
    row_s = pd.Series(row)
    row_s.name = i
    df = df.append(row_s)
df

,id,id_1,id_2,text_1,text_2,jaccard,class
0,1,201,8159,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0.65,0
1,2,202,8158,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0.5,0
2,3,273,8167,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0.611429,0
3,4,220,8160,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,0.324037,-1
4,5,223,8160,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,0.606218,0
...,...,...,...,...,...,...,...
7222,25514,34622,34633,Путин освободил от должности ряд генералов,Путин снял с должностей более 20 руководителей...,0.4624999999999999,0
7223,25524,34566,34654,Облака над Москвой в День Победы разгонят девя...,Путеводитель по Дню Победы: как провести 9 мая...,0.45714285714285713,-1
7224,25548,34519,34681,Любляна отпразднует День Победы вместе с Москвой,В Москве ограничат движение в связи с Днем Победы,0.5842373946721772,-1
7225,25549,34565,34681,Девять самолетов ВВС разгонят облака над Москв...,В Москве ограничат движение в связи с Днем Победы,0.46188021535170065,-1


In [9]:
#Создать словарь
#Ключ: слово
#Значение: порядковый номер

list_text = list(df['text_1']) + list(df['text_2'])

dict_words = {}
i = 0
morpher = MorphAnalyzer()
for text in list_text:
  #разбить предложения на слова
    words = preprocess_txt(morpher, text)
    for word in words:
        if word in dict_words:
          pass
        else:
          dict_words[word] = int(i)
          i += 1
len(dict_words)

11165

In [10]:
text_vocabulary_size = len(dict_words.keys())#10
question_vocabulury_size = len(dict_words.keys())#10000
#answer_vocabulary_size = 500

In [11]:
text_input = Input(shape=(None,), dtype='int32', name='text')

embedded_text = layers.Embedding(
text_vocabulary_size, 64)(text_input)

encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None,),
                      dtype = 'int32',
                      name='question')

embedded_question = layers.Embedding(
text_vocabulary_size, 64)(question_input)

encoded_question = layers.LSTM(32)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

In [12]:
answer = layers.Dense(3, activation='softmax')(concatenated) 

In [13]:
model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [14]:
import numpy as np
morpher = MorphAnalyzer()
num_samples = len(df) #длина датасета
max_length = 100

#text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))

#не забудь нули добавлять в НАЧАЛО
text = np.zeros(shape=(num_samples, max_length))
list_text1 = list(df['text_1'])

for n in range(num_samples):
    words = preprocess_txt(morpher, list_text1[n])
    cur_text = text[n]
    #надо из конца в начало
    i = len(words) - 1
    j = len(cur_text) - 1
    while i >= 0:
        cur_text[j] = dict_words[words[i]]
        i -= 1
        j -= 1
print(text)
print(text.shape)

#question = np.random.randint(1, question_vocabulury_size, size=(num_samples, max_length))
question = np.zeros(shape=(num_samples, max_length))
list_text2 = list(df['text_2'])

for n in range(num_samples):
    words = preprocess_txt(morpher, list_text1[n])
    cur_text = question[n]
    #надо из конца в начало
    i = len(words) - 1
    j = len(cur_text) - 1
    while i >= 0:
        cur_text[j] = dict_words[words[i]]
        i -= 1
        j -= 1
print(question)
print(question.shape)



answers = np.zeros(shape=(num_samples, 3))
indices = np.array(df['class']) #-1,0,1
indices2 = []
for i in indices:
  indices2.append(int(i) + 1) 
   
indices = np.array(indices2)
for i, x in enumerate(answers):
    x[indices[i]] = 1

print(answers)
print(answers.shape)


#model.fit([text, question], answers, epochs=10, batch_size=128)    

[[0.000e+00 0.000e+00 0.000e+00 ... 6.000e+00 7.000e+00 8.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 ... 1.200e+01 1.300e+01 1.400e+01]
 [0.000e+00 0.000e+00 0.000e+00 ... 1.100e+01 2.000e+01 2.100e+01]
 ...
 [0.000e+00 0.000e+00 0.000e+00 ... 5.130e+03 7.000e+00 3.000e+01]
 [0.000e+00 0.000e+00 0.000e+00 ... 1.100e+01 7.000e+01 3.204e+03]
 [0.000e+00 0.000e+00 0.000e+00 ... 1.940e+02 2.109e+03 8.776e+03]]
(7227, 100)
[[0.000e+00 0.000e+00 0.000e+00 ... 6.000e+00 7.000e+00 8.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 ... 1.200e+01 1.300e+01 1.400e+01]
 [0.000e+00 0.000e+00 0.000e+00 ... 1.100e+01 2.000e+01 2.100e+01]
 ...
 [0.000e+00 0.000e+00 0.000e+00 ... 5.130e+03 7.000e+00 3.000e+01]
 [0.000e+00 0.000e+00 0.000e+00 ... 1.100e+01 7.000e+01 3.204e+03]
 [0.000e+00 0.000e+00 0.000e+00 ... 1.940e+02 2.109e+03 8.776e+03]]
(7227, 100)
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
(7227, 3)


In [15]:
from sklearn.model_selection import train_test_split
#text_train, text_val = train_test_split(
#    text, train_size=0.7, random_state=27, shuffle=True
#)
#question_train, question_val = train_test_split(
#    question, train_size=0.7, random_state=27, shuffle=True
#)

#answers_train, answers_val = train_test_split(
#    answers, train_size=0.7, random_state=27, shuffle=True
#)

training_samples = 6200
validation_samples = 1000
text_train  = text[:training_samples]
question_train = question[:training_samples]
answer_train = answers[:training_samples]
text_val = text[training_samples: training_samples + validation_samples]
question_val = question[training_samples: training_samples + validation_samples]
answer_val = answers[training_samples: training_samples + validation_samples]

In [16]:
history = model.fit([text_train, question_train], answer_train, epochs=3, batch_size=128, validation_data=([text_val,question_val], answer_val))    

Epoch 1/3
49/49 [==============================] - 13s 173ms/step - loss: 1.0646 - acc: 0.4347 - val_loss: 1.0863 - val_acc: 0.3780
Epoch 2/3
49/49 [==============================] - 7s 152ms/step - loss: 0.9660 - acc: 0.5421 - val_loss: 1.1275 - val_acc: 0.3960
Epoch 3/3
49/49 [==============================] - 7s 153ms/step - loss: 0.8289 - acc: 0.6340 - val_loss: 1.2327 - val_acc: 0.3990


In [17]:
results = model.evaluate([text_val,question_val], answer_val)

32/32 [==============================] - 1s 21ms/step - loss: 1.2327 - acc: 0.3990


In [18]:
results

[1.2327274084091187, 0.39899998903274536]

In [19]:
y_hat = model.predict([text_val, question_val])

In [20]:
y_hat.shape

(1000, 3)

In [21]:
from sklearn.metrics import roc_auc_score, roc_curve
pct_auc = roc_auc_score(answer_val[:,0], y_hat[:,0])*100
print("ROC AUC по несхожим фразам: ", pct_auc)

ROC AUC по несхожим фразам:  57.097934081127676


In [22]:
pct_auc = roc_auc_score(answer_val[:,1], y_hat[:,1])*100
print("ROC AUC по схожим фразам: ", pct_auc)

ROC AUC по схожим фразам:  53.98155298155298


In [23]:
pct_auc = roc_auc_score(answer_val[:,2], y_hat[:,2])*100
print("ROC AUC по идентичным фразам: ", pct_auc)

ROC AUC по идентичным фразам:  57.02415317122495


In [24]:
#нужно взять 2 произвольные фразы, оцифровать их и определить похожесть
#
import numpy as np

#функция 
#вход: передаем на вход 2 фразы
#выход: на выходе строка 
#Классы перефразирования: -1: непарафразы, 0: свободные перефразы, 1: строгие перефразы.

def get_paraphrase_classes(phrase1, phrase2):
    morpher = MorphAnalyzer()
    num_samples = 1 #длина датасета
    max_length = 100

    #не забудь нули добавлять в НАЧАЛО
    text = np.zeros(shape=(num_samples, max_length))

    words = preprocess_txt(morpher, phrase1)
    print(words)
    cur_text = text[0]
    #надо из конца в начало
    i = len(words) - 1
    j = len(cur_text) - 1
    while i >= 0:
    #проверка на наличие слова в словаре
        if words[i] in dict_words.keys(): 
            cur_text[j] = dict_words[words[i]]
        else:
            cur_text[j] = len(dict_words.keys()) + 1    
        i -= 1
        j -= 1
#----------------------------
#не забудь нули добавлять в НАЧАЛО
    question = np.zeros(shape=(num_samples, max_length))

    words = preprocess_txt(morpher, phrase2)
    cur_text = question[0]
    #надо из конца в начало
    i = len(words) - 1
    j = len(cur_text) - 1
    while i >= 0:
    #проверка на наличие слова в словаре
        if words[i] in dict_words.keys(): 
            cur_text[j] = dict_words[words[i]]
        else:
            cur_text[j] = len(dict_words.keys()) + 1    
        i -= 1
        j -= 1

    predictions = model.predict([text, question])
    n = np.argmax(predictions[i,:])
#Paraphrase classes: -1: non-paraphrases, 0: loose paraphrases, 1: strict paraphrases.
#Классы перефразирования: -1: непарафразы, 0: свободные перефразы, 1: строгие перефразы.
    if n == 0:
        return False
    else:
        return True    

In [27]:
phrase1 = "Полицейским разрешат стрелять на поражение по гражданам с травматикой."
phrase2 = "Полиции могут разрешить стрелять по хулиганам с травматикой."

paraphrase_class = get_paraphrase_classes(phrase1, phrase2)
if paraphrase_class:
    print("свободные или строгие перефразы")
else:
    print('не являются перефразами')       

['полицейский', 'разрешить', 'стрелять', 'на', 'поражение', 'по', 'гражданин', 'с', 'травматика']
свободные или строгие перефразы
